In [1]:
#PREPARE_gse_network_metabolomics       22.04.18
#although techincally, it should not be "GSE" since this is just a hypergeometric test with pathways.
#But I am naming this as gse for convenience

#prepare network (cytoscape format) topology and information file.
#node: 1 (DEG group), 2 (pathway group)
#node size: (number of elements)
#edge: width (number of elements)

#output1: topology file (contains node size)
#output2: edge information file


import pandas as pd

In [73]:
def update_data_dict(ready_file, result_file, data_dict, node_info_dict, pathway_list, condition_regulation_list, condition, regulation):

    result_df = pd.read_csv(result_file, sep="\t")
    r, c = result_df.shape  
    temp_pathway_list = []
    
    #identify which pathway is significant
    for i in range(r):
        pathway = result_df["pathway"][i]
        pvalue = result_df["pvalue"][i]
        
        if pvalue < 0.05:
            temp_pathway_list.append(pathway)
            
    ready_df = pd.read_csv(ready_file, sep="\t")
    r, c = ready_df.shape
      
    #update data_dict, pathway_list: significant pathways, get mvalue, kvalue
    for i in range(r):
        pathway = ready_df["pathway"][i]
        x_value = ready_df["significant_metabolites_in_pathway(x)"][i]
        k_value = ready_df["detected_significant_metabolites(k)"][i]
        condition_regulation = '%s_%s' % (condition, regulation)
                           
        if pathway in temp_pathway_list:
            pathway_list.append(pathway)    
            condition_regulation_list.append(condition_regulation)
            data_dict[pathway, condition_regulation] = x_value
            node_info_dict[pathway] = k_value
            
        if condition_regulation not in list(node_info_dict.keys()):
            node_info_dict[condition_regulation] = k_value
                           
    return data_dict, node_info_dict, pathway_list, condition_regulation_list

In [91]:
data_dir = "../../../analysis/statistics/linear_model_no_filter/analysis/metabolomics"
output_dir = "../../../analysis/statistics/linear_model_no_filter/analysis/metabolomics/gse_network"

condition_list = ['cVSneg','cVSpos','cVSra','negVSpos']
updown_list = ['down','up']

data_dict = {}
pathway_list = []
condition_regulation_list = []
node_info_dict = {}

for condition in condition_list:
    for regulation in updown_list:
        gse_ready_file = '%s/metabolomics.%s.%s.hypergeometric.ready.tsv' % (data_dir, condition, regulation)
        gse_result_file = '%s/metabolomics.%s.%s.hypergeometric.result.tsv' % (data_dir, condition, regulation)
        data_dict, node_info_dict, pathway_list, condition_regulation_list = update_data_dict(gse_ready_file, gse_result_file, data_dict, node_info_dict, pathway_list, condition_regulation_list, condition, regulation)
        

pathway_list = list(set(pathway_list))
condition_regulation_list = list(set(condition_regulation_list))        
    
output_topology_txt = open('%s/metabolomics.topology.tsv' % output_dir ,'w')
output_info_txt = open('%s/metabolomics.topology.info.tsv' % output_dir ,'w')

output_topology_txt.write('source\ttarget\tedge_size\n')
for condition_regulation in condition_regulation_list:
        for pathway in pathway_list:       
            try: 
                xvalue = data_dict[pathway, condition_regulation]
                output_topology_txt.write('%s\t%s\t%s\n' % (condition_regulation, pathway, xvalue))
            except KeyError:
                None
            
output_info_txt.write('node\tnodesize\tnode_type\n')
for condition_regulation in condition_regulation_list:
    node_size = node_info_dict[condition_regulation]
    output_info_txt.write('%s\t%s\t1\n' % (condition_regulation, node_size))

for pathway in pathway_list:
    node_size = node_info_dict[pathway]
    output_info_txt.write('%s\t10\t2\n' % (pathway))

output_topology_txt.close()
output_info_txt.close()
        